In [4]:
import xarray as xr
import numpy as np
#import pandas as pd
import polars as pl
import math
# import time
# import orjson
from geopy.distance import geodesic
from fastapi import FastAPI, status  # , Depends
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from typing import Union
import dask

In [143]:
def gridded_arcsec(x, base=90, arc=3600/15):
    idx = (int(x) + base)* arc + math.ceil((x-int(x))*arc)
    return idx-1 if idx > 0 else 0

def curDist(loc, dis=np.empty(shape=[0, 1], dtype=float)):
    if len(loc) < 2:
        return 0  # None
    lk = len(loc)-1
    return(np.append(dis,
                     geodesic((loc[lk-1, 1], loc[lk-1, 0]),
                              (loc[lk, 1], loc[lk, 0])).km))


In [237]:
arcsec = 15
arc = int(3600/arcsec)  # 15 arc-second
basex = 180  # -180 - 180 <==> 0 - 360, half is 180
basey = 90  # -90 - 90 <==> 0 - 180, half is 90
halfxidx = 180 * arc  # in netcdf, longitude length = 86400
halfyidx = 90 * arc  # in netcdf, latitude length = 43200
subsetFlag = True
#zmode = "line" #"point"
#format = "row"

# forward, cross-zero
# lonx = np.array([-0.153459759571552,-0.0984716760636,0.14980816495167, -0.03]) 
# latx = np.array([-0.163806258890185,-0.074795994003052,0.264, 0.274])
# one-point closer to zero
# lonx = np.array([-0.153459759571552,-0.0020816760636,0.14980816495167, -0.00207]) 
# latx = np.array([-0.163806258890185,-0.074795994003052,0.264, 0.274])
# two-points closer to zero
# lonx = np.array([-0.153459759571552,-0.0020816760636, 0.00207]) 
# latx = np.array([-0.163806258890185,-0.074795994003052,0.164])

# backward, away-zero
#lonx = np.array([179.8465,179.9015,-179.8502, 179.88]) 
#latx = np.array([0.163806258890185, 0.074795994003052, -0.264, -0.274])
# one-point closer to 180-line
#lonx = np.array([179.8465,179.9985,-179.9902, 179.88, -179.9983]) 
#latx = np.array([0.163806,0.07479599, -0.264, -0.274, -0.166])

#normal case
# lonx = np.array([122.26981235231769,122.46580467738616,122.52322927209325])
# latx = np.array([24.53014834691002,24.484704345640385,24.43356023689588])
# aircraft runway
lonx = np.array([121.53967066033685,121.5649924414543])
latx = np.array([25.070035818788032,25.06911681954908])

In [201]:
# want to solve a situation when cross 180-line or zero-line (prime meridian)
l1a = [-179.95, 25]
l2a = [179,95, 24]
l1b = [-0.05, 25]
l2b = [0.05, 24]

print(geodesic((25,-179.95), (25,179.95)).km)
print(geodesic((25,-0.05), (25,0.05)).km)

def whichSide(l1a, l2a): #, direction='auto'):
   #autoFly = 'cross-zero' #cross prime meridian #otherwise, "away-zero" (cross 180-degree)
   ##(deprecated) dirFlag: get direction (clockwise or counter-) when autoFly
   if l1a[0] != l2a[0] and np.sign(l1a[0]) != np.sign(l2a[0]):
      lonat1 = 180 + l1a[0]
      lonat2 = 180 + l2a[0]
      dift1 = np.absolute(lonat2 - lonat1)

      lonas1 = 180 - l1a[0] if l1a[0]>=0 else -180 - l1a[0]
      lonas2 = 180 - l2a[0] if l2a[0]>=0 else -180 - l2a[0]
      dift2 = np.absolute(lonas2 - lonas1) 

      if dift1 < dift2:
         #dirFlag = 'clockwise' if np.sign(l1a[0]) == 1 else 'counterclockwise' 
         autoFly = 'cross-zero'
      else:
         #dirFlag = 'counterclockwise' if np.sign(l1a[0]) == 1 else 'clockwise' 
         autoFly = 'away-zero'

      #print("AutoFly: ", autoFly, dirFlag)
      return(autoFly) #, dirFlag)
   else:
      return(None) #, None)


print(whichSide(l1a, l2a))
print(whichSide(l1b, l2b))
print(whichSide([35.95, 25], [-35, 24]))
print(whichSide([-130.95, 25], [35, 24]))
tt = whichSide([35.95, 25], [-160, 24])
print(tt) #[0], tt[1])



10.095008799533332
10.095008799531035
away-zero
cross-zero
cross-zero
cross-zero
away-zero


In [188]:
def getlinex(lon, lat):
    if (lon[1]==lon[0]):
        return(None, None)
    if (lat[1]==lat[0]):
        return(0, lat[0])
    m = (lat[1]-lat[0])/(lon[1]-lon[0])
    b = lat[0] - m * lon[0]  # y = mx + b
    return(m, b)

In [231]:
def crossBoundary(lon, lat, debug=False):
    if len(lon) < 2:
        return(lon, lat, [-1], []) #, []) #lon, lat, idx, autoFly, (deprecated) dirFlag
    
    idx = np.ravel(np.where(np.sign(lon[:-1]) != np.sign(lon[1:])))
    if len(idx) == 0:
        return(lon, lat, [-1], []) # , [])
    
    basex=180
    #zeroidx=43200
    zeroloc=0.5*15/3600
    autoFly = np.empty(shape=[0, 1], dtype=str)
    #dirFlag = np.empty(shape=[0, 1], dtype=str)
    newx = np.empty(shape=[0, 1], dtype=float)
    newy = np.empty(shape=[0, 1], dtype=float)
    newidx = np.empty(shape=[0, 1], dtype=int) #recalculate break-points index in newx
    preidx = 0
    for k in range(len(idx)):
        idx0 = idx[k] #.item()
        idx1 = -1 if k == len(idx)-1 else idx[k+1] #.item()
        newx = np.append(newx, lon[preidx:idx0+1], axis=None)
        newy = np.append(newy, lat[preidx:idx0+1], axis=None)
        tf = whichSide([lon[idx0]], [lon[idx0+1]])
        autoFly = np.append(autoFly, tf, axis=None) #tf[0]
        #dirFlag = np.append(dirFlag, tf[1], axis=None)
        if tf == 'cross-zero':
            m = (lat[idx1]-lat[idx0])/(lon[idx1]-lon[idx0])
            b = lat[idx0] - m * lon[idx0]  # y = mx + b
            absm = np.absolute(m) # if m > 1, even two-ends closer to zero, we still need append points 
            absxdelta = np.absolute((0.499/arc)/m) #if larger m > 1, need a smaller step near 0-line
            #mb = getlinex([lon[idx0], lon[idx1]], [lat[idx0], lat[idx1]])
            if np.sign(newx[-1]) == -1: 
                if absm <= 1 and newx[-1] >= -zeroloc and lon[idx0+1] <= zeroloc: #i.e, both-ends closer to zero
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks in newx[-1], but no new x needed to insert
                elif absm <= 1 and newx[-1] >= -zeroloc: 
                    newx = np.append(newx, zeroloc, axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks before zeroloc 
                    newy = np.append(newy, m * zeroloc + b, axis=None)
                elif absm <= 1 and lon[idx0+1] <= zeroloc: 
                    newx = np.append(newx, -zeroloc, axis=None)
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks after -zeroloc 
                    newy = np.append(newy, m * (-zeroloc) + b, axis=None)
                else:
                    if absm > 1:
                        zeroloc = absxdelta
                    leftx = -zeroloc if newx[-1] < -zeroloc else newx[-1]*0.1 #a much closer-zero value but still in negative side
                    rightx= zeroloc if lon[idx0+1] > zeroloc else lon[idx0+1]*0.1
                    zeroin = [leftx, rightx]
                    newx = np.append(newx, zeroin, axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks in -zeroloc - zeroloc
                    newy = np.append(newy, [m * zeroin[0] + b, m * zeroin[1] + b], axis=None)
            else: #np.sign(newx[-1]) == 1:
                if absm <= 1 and newx[-1] <= zeroloc and lon[idx0+1] >= -zeroloc: #i.e, both-ends closer to zero
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks in newx[-1], but no new x needed to insert
                elif absm <= 1 and newx[-1] <= zeroloc: 
                    newx = np.append(newx, -zeroloc, axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks before -zeroloc 
                    newy = np.append(newy, m * (-zeroloc) + b, axis=None)
                elif absm <= 1 and lon[idx0+1] >= -zeroloc: 
                    newx = np.append(newx, zeroloc, axis=None)
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks after zeroloc 
                    newy = np.append(newy, m * zeroloc + b, axis=None)
                else:
                    if absm > 1:
                        zeroloc = absxdelta
                    leftx = zeroloc if newx[-1] > zeroloc else newx[-1]*0.1 #a much closer-zero value but still in positive side
                    rightx= -zeroloc if lon[idx0+1] < -zeroloc else lon[idx0+1]*0.1
                    zeroin = [leftx, rightx]
                    newx = np.append(newx, zeroin, axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks in zeroloc - -zeroloc
                    newy = np.append(newy, [m * zeroin[0] + b, m * zeroin[1] + b], axis=None)
        else:
            endat = [-(basex-zeroloc), basex-zeroloc] if np.sign(newx[-1]) == -1 else [basex-zeroloc, -(basex-zeroloc)]
            # Note: -179 <--> 179 is a big disrupt, cannot use to calculate y = mx + b
            # use formula in function whichSide() 
            lonas0 = basex - lon[idx0] if lon[idx0]>=0 else -basex - lon[idx0]
            lonas1 = basex - lon[idx1] if lon[idx1]>=0 else -basex - lon[idx1] 
            #print([lon[idx0], lat[idx0]], [lon[idx1], lat[idx1]])
            #print("rewrite lon: ", lonas0, lonas1)
            m = (lat[idx1]-lat[idx0])/(lonas1-lonas0) 
            b = lat[idx0] - m * lonas0
            lonat0 = basex - endat[0] if endat[0]>=0 else -basex - endat[0]
            lonat1 = basex - endat[1] if endat[1]>=0 else -basex - endat[1] 
            absm = np.absolute(m)
            absxdelta = np.absolute((0.499/arc)/m) 
            #print("m, b: ", m, b)
            #print("new x: ", endat) 
            #print("new y: ", m * lonat0 + b, m * lonat1 + b)
            if np.sign(newx[-1]) == -1: 
                if absm <= 1 and newx[-1] <= -(basex-zeroloc) and lon[idx0+1] >= basex-zeroloc: #i.e, both-ends closer to 180-line
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks in newx[-1], but no new x needed to insert
                elif absm <= 1 and newx[-1] <= -(basex-zeroloc): 
                    newx = np.append(newx, basex-zeroloc, axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks before 180-zeroloc
                    lonat1 = basex - endat[1] if endat[1]>=0 else -basex - endat[1] #shold positive 
                    newy = np.append(newy, m * lonat1 + b, axis=None)
                elif absm <= 1 and lon[idx0+1] >= basex-zeroloc: 
                    newx = np.append(newx, -(basex-zeroloc), axis=None)
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks after -(180-zeroloc)
                    lonat0 = basex - endat[0] if endat[0]>=0 else -basex - endat[0]
                    newy = np.append(newy, m * lonat0 + b, axis=None)
                else:
                    if absm > 1:
                        zeroloc = absxdelta
                    leftx = -(basex-zeroloc) if newx[-1] > -(basex-zeroloc) else 0.5*(-basex+newx[-1]) #a much closer-180 value but still in negative side
                    rightx= basex-zeroloc if lon[idx0+1] < basex-zeroloc else 0.5*(basex+lon[idx0+1])
                    endat = [leftx, rightx]
                    lonat0 = basex - endat[0] if endat[0]>=0 else -basex - endat[0]
                    lonat1 = basex - endat[1] if endat[1]>=0 else -basex - endat[1] 
                    newx = np.append(newx, endat, axis=None)
                    newy = np.append(newy, [m * lonat0 + b, m * lonat1 + b], axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks in -endat, endat
            else: #np.sign(newx[-1]) == 1:
                if absm <= 1 and newx[-1] >= basex-zeroloc and lon[idx0+1] <= -(basex-zeroloc): #i.e, both-ends closer to 180-line
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks in newx[-1], but no new x needed to insert
                elif absm <= 1 and newx[-1] >= basex-zeroloc: 
                    newx = np.append(newx, -(basex-zeroloc), axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks before -(180-zeroloc) 
                    lonat1 = basex - endat[1] if endat[1]>=0 else -basex - endat[1] #shold negative
                    newy = np.append(newy, m * lonat1 + b, axis=None)
                elif absm <= 1 and lon[idx0+1] <= -(basex-zeroloc): 
                    newx = np.append(newx, basex-zeroloc, axis=None)
                    newidx = np.append(newidx, len(newx)-1, axis=None) #breaks after basex-zeroloc 
                    lonat0 = basex - endat[0] if endat[0]>=0 else -basex - endat[0]
                    newy = np.append(newy, m * lonat0 + b, axis=None)
                else:
                    if absm > 1:
                        zeroloc = absxdelta
                    leftx = basex-zeroloc if newx[-1] < basex-zeroloc else 0.5*(basex+newx[-1]) #a much closer-180 value but still in positive side
                    rightx= -(basex-zeroloc) if lon[idx0+1] > -(basex-zeroloc) else 0.5*(-basex+lon[idx0+1])
                    endat = [leftx, rightx]
                    lonat0 = basex - endat[0] if endat[0]>=0 else -basex - endat[0]
                    lonat1 = basex - endat[1] if endat[1]>=0 else -basex - endat[1] 
                    newx = np.append(newx, endat, axis=None)
                    newy = np.append(newy, [m * lonat0 + b, m * lonat1 + b], axis=None)
                    newidx = np.append(newidx, len(newx)-2, axis=None) #breaks in -endat, endat
            
        preidx = idx0+1

    newx = np.append(newx, lon[preidx:], axis=None)                 
    newy = np.append(newy, lat[preidx:], axis=None)
    newidx = np.append(newidx, -1, axis=None)
    if debug:
        df = pl.DataFrame({"longitude": newx, "latitude": newy})
        df.write_csv("simu/test_loc.csv")                 
    return(newx, newy, newidx, autoFly) # , dirFlag)     


#a = np.array([1,1,-1,-2,-3,4,5])
#idx1 = np.where(np.sign(a[:-1]) != np.sign(a[1:]))[0] 
#print(idx1)
print(crossBoundary(lonx, latx, True))

(array([ 179.8465   ,  179.9015   ,  179.9984762, -179.9984762,
       -179.8502   , -179.9984762,  179.9984762,  179.88     ]), array([ 0.16380626,  0.07479599, -0.05752438, -0.06168271, -0.264     ,
       -0.26949578, -0.26960874, -0.274     ]), array([ 2,  5, -1]), array(['away-zero', 'away-zero'], dtype='<U9'))


In [238]:
def zprofile_cross(loni: str, lati: str, mode: Union[str, None] = None):
    #lonx = numarr_query_validator(lon)
    #latx = numarr_query_validator(lat)
    #if isinstance(lonx, str) or isinstance(latx, str):
    #    return JSONResponse(status_code=status.HTTP_400_BAD_REQUEST,
    #                        content=jsonable_encoder({"Error": "Check your input format should be comma-separated values"}))

    global ds
    global arcsec
    global arc
    global basex
    global basey
    global halfxidx
    global halfyidx
    global subsetFlag
    ds = xr.open_zarr(
        'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', group='gebco',
        decode_cf=False, decode_times=False)

    # st = time.time()
    # ds = xr.open_mfdataset(
    #    'GEBCO_2022_sub_ice_topo.nc', chunks={0: 60, 1: 60},
    # et = time.time()
    # print('1. Load dataset time: ', et-st, 'sec')
    # lon = [123.98, 123]  # just testing
    # lat = [23.33, 21.33]
    # i.e. {longitude=[...], laitude=[...], z=[...]}, otherwise 'row', output df-like records
    format = 'default'
    # i.e, output all gridded points along the line; otherwise 'point', output only end-points.
    zmode = 'line'
    if mode is not None:
        if 'point' in mode.lower():
            zmode = 'point'
        if 'row' in mode.lower():
            format = 'row'
    #print("current mode: ", format, " & ", zmode)

    if len(loni) != len(lati):
        ds.close()
        return JSONResponse(status_code=status.HTTP_400_BAD_REQUEST,
                            content=jsonable_encoder({"Error": "Check your input of lon/lat should be in equal length"}))

    # May not have exact lon, lat in gridded lon-lat, so need to calculate index
    elif len(loni) == 1:
        # st = time.time()
        lon0 = gridded_arcsec(loni[0], basex, arc)
        lat0 = gridded_arcsec(lati[0], basey, arc)
        mlon0x = ds["lon"][lon0]
        mlat0x = ds["lat"][lat0]
        st1 = ds.sel(lon=mlon0x, lat=mlat0x)
        ds.close()
        loc1 = [loni[0], lati[0]]
        xt1 = np.array([st1['elevation'].values])
        if format == 'row':
            # df1= pd.DataFrame({"longitude": np.array([loc1[0]]).tolist(),
            df1 = pl.DataFrame({"longitude": np.array([loc1[0]]).tolist(),
                                "latitude": np.array([loc1[1]]).tolist(),
                                "z": xt1.tolist(), "distance": np.array([0]).tolist()},
                               columns=['longitude', 'latitude', 'z', 'distance'])
        else:
            out = jsonable_encoder({"longitude": np.array([loc1[0]]).tolist(),
                                    "latitude": np.array([loc1[1]]).tolist(),
                                    "z": xt1.tolist(), "distance": np.array([0]).tolist()})
        # et = time.time()
        # print('Get only one-point exe time: ', et-st, 'sec')
        # return(xt1)
    else:
        # st = time.time()
        if zmode == 'point':
            lonk = loni
            latk = lati
            brks = [-1]
        else:    
            ats = crossBoundary(loni, lati)
            lonk = np.asarray(ats[0])
            latk = np.asarray(ats[1])
            brks = ats[2]
            #autoFly = ats[3]
            #dirFlag = ats[4] #deprecated
        #print(lonk, latk, brks)
        idx1 = np.empty(shape=[0, 2], dtype=np.int16)
        loc1 = np.empty(shape=[0, 2], dtype=float)
        dis1 = np.array([0.0])
        mlon0 = np.min(lonk) - 1.5/arc   # to make it smaller
        mlon0 = mlon0 if mlon0 > -basex else -basex+0.00001
        #mlon1 = np.max(long)
        mlat0 = np.min(latk) - 1.5/arc
        mlat0 = mlat0 if mlat0 > -basey else -basey+0.00001
        #mlat1 = np.max(latg)
        # if do subsetting dataset, reference-0-x,y should be biased
        mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0
        mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
        #print("base: ", mlonbase, mlatbase, mlon0, mlat0)
        # index from gridded_arcsec will change is reference is ds_s1, not ds
        #print(mlonbase, mlatbase)
        #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
        #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
        preidx = 0 
        for brk in brks:
            print(brk)
            lonx = lonk[preidx:] if brk == -1 else lonk[preidx:(brk+1)]
            latx = latk[preidx:] if brk == -1 else latk[preidx:(brk+1)]
            print(lonx, latx)
            if len(lonx) == 1: #only occur because break-points cause a closer-zero end-pt appear in last segment 
                lonidx0 = gridded_arcsec(lonx[0], basex, arc)
                latidx0 = gridded_arcsec(latx[0], basey, arc)
                idx1 = np.append(
                    idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lonx[0], latx[0]]], axis=0)
                continue

            for i in range(len(lonx)-1):
                lonidx0 = gridded_arcsec(lonx[i], basex, arc)
                latidx0 = gridded_arcsec(latx[i], basey, arc)
                lonidx1 = gridded_arcsec(lonx[i+1], basex, arc)
                latidx1 = gridded_arcsec(latx[i+1], basey, arc)
                #print("check i: ", i, [lonx[i], lonx[i+1]], lonidx0, lonidx1,
                #      " get lon: ", ds["lon"][lonidx0].item(), ds["lon"][lonidx0+1].item(), ds["lon"][lonidx0+2].item())
                if zmode == 'point':
                    idx1 = np.append(
                        idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
                    dist = geodesic((latx[i], lonx[i]),
                                    (latx[i+1], lonx[i+1])).km
                    dis1 = np.append(dis1, dist, axis=None)
                    if i == len(lonx)-2:
                        idx1 = np.append(
                            idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lonx[i+1], latx[i+1]]], axis=0)
                elif lonidx0 == lonidx1 and latidx0 == latidx1:
                    idx1 = np.append(
                        idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
                    # to match the same length
                    if i >= 1:
                        dis1 = curDist(loc1, dis1)
                else:
                    if lonx[i] == lonx[i+1]:
                        stepi = -1 if latidx0 > latidx1 else 1
                        rngi = range(latidx0, latidx1+stepi, stepi)
                        leni = len(rngi)
                        for k, y in enumerate(rngi):
                            locy0 = (y+1)/arc - basey
                            locy0i = int(locy0)
                            doty0i = locy0-locy0i-0.25/arc  # a small bias to make sure it's in grid
                            locy1 = locy0i + doty0i
                            locy1 = basey if locy1 > basey else (-basey if locy1 < -basey else locy1)
                            if (k < (leni-1)) or (stepi == 1 and locy1 < latx[i+1]) or (stepi == -1 and locy1 > latx[i+1]):
                                idx1 = np.append(
                                    idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                                loc1 = np.append(loc1, [[lonx[i], locy1]], axis=0)
                                if i >= 1 or k >= 1:
                                    dis1 = curDist(loc1, dis1)

                    elif latx[i] == latx[i+1]:
                        stepi = -1 if lonidx0 > lonidx1 else 1
                        rngi = range(lonidx0, lonidx1+stepi, stepi)
                        leni = len(rngi)
                        for k, x in enumerate(rngi):
                            locx0 = (x+1)/arc - basex
                            locx0i = int(locx0)
                            dotx0i = locx0-locx0i-0.25/arc  # a small bias to make sure it's in grid
                            locx1 = locx0i + dotx0i
                            locx1 = basex if locx1 > basex else (-basex if locx1 < -basex else locx1)
                            if (k < (leni-1)) or (stepi == 1 and locx1 < lonx[i+1]) or (stepi == -1 and locx1 > lonx[i+1]):
                                idx1 = np.append(
                                    idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                                loc1 = np.append(loc1, [[locx1, latx[i]]], axis=0)
                                if i >= 1 or k >= 1:
                                    dis1 = curDist(loc1, dis1)

                    else:
                        m = (latx[i+1]-latx[i])/(lonx[i+1]-lonx[i])
                        b = latx[i] - m * lonx[i]  # y = mx + b
                        print("m, b:", m ,b)
                        if np.absolute(m) <= 1:
                            lidx0 = lonidx0
                            lidx1 = lonidx1
                        else:
                            lidx0 = latidx0
                            lidx1 = latidx1

                        stepi = -1 if lidx0 > lidx1 else 1
                        rngi = range(lidx0, lidx1+stepi, stepi)
                        leni = len(rngi)
                        for k, s in enumerate(rngi):
                            if k == 0:  # should consider internal node not repeated twice
                                idx1 = np.append(
                                    idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                                loc1 = np.append(
                                    loc1, [[lonx[i], latx[i]]], axis=0)
                                if i >= 1:
                                    dis1 = curDist(loc1, dis1)

                            else:
                                if np.absolute(m) <= 1:
                                    locx0 = (s+1)/arc - basex
                                    locx0i = int(locx0)
                                    dotx0i = locx0-locx0i-0.25/arc  # a small bias to make sure it's in grid
                                    locx1 = locx0i + dotx0i
                                    locx1 = basex if locx1 > basex else (-basex if locx1 < -basex else locx1)
                                    #print("check k: ", k, s, locx0, locx1)
                                    locy1 = m * locx1 + b
                                    locy1 = basey if locy1 > basey else (-basey if locy1 < -basey else locy1)
                                    if (k < (leni-1)) or (stepi == 1 and locx1 < lonx[i+1]) or (stepi == -1 and locx1 > lonx[i+1]):
                                        y = gridded_arcsec(locy1, basey, arc)
                                        idx1 = np.append(
                                            idx1, [[y-mlatbase, s-mlonbase]], axis=0)
                                        loc1 = np.append(
                                            loc1, [[locx1, locy1]], axis=0)
                                        dis1 = curDist(loc1, dis1)
                                else:
                                    locy0 = (s+1)/arc - basey
                                    locy0i = int(locy0)
                                    doty0i = locy0-locy0i-0.25/arc
                                    locy1 = locy0i + doty0i
                                    locy1 = basey if locy1 > basey else (-basey if locy1 < -basey else locy1)
                                    locx1 = (locy1 - b)/m
                                    locx1 = basex if locx1 > basex else (-basex if locx1 < -basex else locx1)
                                    if (k < (leni-1)) or (stepi == 1 and locy1 < latx[i+1]) or (stepi == -1 and locy1 > latx[i+1]):
                                        x = gridded_arcsec(locx1, basex, arc)
                                        idx1 = np.append(
                                            idx1, [[s-mlatbase, x-mlonbase]], axis=0)
                                        loc1 = np.append(
                                            loc1, [[locx1, locy1]], axis=0)
                                        dis1 = curDist(loc1, dis1)

                print("check i, len(lonx)", i, len(lonx))
                if zmode != 'point' and i == len(lonx)-2:
                    idx1 = np.append(
                        idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[lonx[i+1], latx[i+1]]], axis=0)
                    dis1 = curDist(loc1, dis1)
            #end loop-i
            #print(idx1)
            #print(loc1)
            #print(dis1)
            if brk != -1:               
                dist = geodesic((latk[brk], lonk[brk]),
                                (latk[brk+1], lonk[brk+1])).km
                dis1 = np.append(dis1, dist, axis=None)            
                preidx = brk+1
        # end loop-brks
        # np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f')
        # st = time.time()
        # mlon0 = np.min(lonx) #may cause slice offset to mlonbase, an offset +-1
        mlon1 = np.max(lonk) + 1.5/arc   # to make it larger
        mlon1 = mlon1 if mlon1 <= basex else basex-0.00001
        # mlat0 = np.min(latx) #may cause slice offset to mlatbase, an offset +-1
        mlat1 = np.max(latk) + 1.5/arc
        mlat1 = mlat1 if mlat1 <= basey else basey-0.00001
        mlon0x = ds["lon"][mlonbase].item()
        mlat0x = ds["lat"][mlatbase].item()
        #print("bbox info:", mlon1, mlat1, mlon0x, mlat0x)
        ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(
            mlat0x, mlat1)) if subsetFlag else ds
        ds.close()
        # print(ds_s1['elevation'].shape)
        #print(idx1)
        xt1 = ds_s1['elevation'].values[tuple(idx1.T)]
        ds_s1.close()
        if format == 'row':
            #print(loc1)
            #print(xt1)
            #print(dis1)
            print(len(loc1[:,0]))
            print(len(xt1))
            print(len(dis1))
            # df1= pd.DataFrame({"longitude": loc1[:, 0].tolist(),
            df1 = pl.DataFrame({"longitude": loc1[:, 0].tolist(),
                                "latitude": loc1[:, 1].tolist(),
                                "z": xt1.tolist(),
                                "distance": dis1.tolist()},
                               columns=['longitude', 'latitude', 'z', 'distance'])
        else:
            out = jsonable_encoder({"longitude": loc1[:, 0].tolist(),
                                    "latitude": loc1[:, 1].tolist(),
                                    "z": xt1.tolist(),
                                    "distance": dis1.tolist()})
        # et = time.time()
        # print('Get value from index of points, time: ', et-st, 'sec')
        # return({"data": xt1})
        # np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

    # st = time.time()
    # jt1 = orjson.dumps(xt1.tolist(), option=orjson.OPT_NAIVE_UTC |
    #                   orjson.OPT_SERIALIZE_NUMPY)
    # out = jsonable_encoder({"data": xt1.tolist()})
    if format == 'row':
        # out = df1.to_dict(orient='records') #by using pandas
        # out = df1.to_dicts()  # by polars
        out = df1
    # et = time.time()
    # print('4 Convert JSON by fastapi: ', et-st, 'sec')
    return out #JSONResponse(content=out)


In [239]:
px1 = zprofile_cross(lonx, latx, mode='row')
px1.select(["longitude","latitude"]).write_csv('simu/test_loc.csv') 
px1.write_csv('simu/test_zseg.csv') 
print(px1)

-1
[122.26981235 122.46580468 122.52322927] [24.53014835 24.48470435 24.43356024]
m, b: -0.23186622870951648 52.88038862206218
check i, len(lonx) 0 3
m, b: -0.8906307307066522 133.55651345203898
check i, len(lonx) 1 3
63
63
63
shape: (63, 4)
┌────────────┬───────────┬──────┬──────────┐
│ longitude  ┆ latitude  ┆ z    ┆ distance │
│ ---        ┆ ---       ┆ ---  ┆ ---      │
│ f64        ┆ f64       ┆ i64  ┆ f64      │
╞════════════╪═══════════╪══════╪══════════╡
│ 122.269812 ┆ 24.530148 ┆ -232 ┆ 0.0      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 122.273958 ┆ 24.529187 ┆ -236 ┆ 0.4334   │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 122.278125 ┆ 24.528221 ┆ -236 ┆ 0.435566 │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 122.282292 ┆ 24.527255 ┆ -250 ┆ 0.435569 │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ ...        ┆ ...       ┆ ...  ┆ ...      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 122.511458 ┆ 24.444044 ┆ -591 ┆ 0.589454 │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌

In [155]:
def zprofile(lonx: str, latx: str, mode: Union[str, None] = None):
    #lonx = numarr_query_validator(lon)
    #latx = numarr_query_validator(lat)
    #if isinstance(lonx, str) or isinstance(latx, str):
    #    return JSONResponse(status_code=status.HTTP_400_BAD_REQUEST,
    #                        content=jsonable_encoder({"Error": "Check your input format should be comma-separated values"}))

    global ds
    global arcsec
    global arc
    global basex
    global basey
    global halfxidx
    global halfyidx
    global subsetFlag
    ds = xr.open_zarr(
        'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', group='gebco',
        decode_cf=False, decode_times=False)

    # st = time.time()
    # ds = xr.open_mfdataset(
    #    'GEBCO_2022_sub_ice_topo.nc', chunks={0: 60, 1: 60},
    # et = time.time()
    # print('1. Load dataset time: ', et-st, 'sec')
    # lon = [123.98, 123]  # just testing
    # lat = [23.33, 21.33]
    # i.e. {longitude=[...], laitude=[...], z=[...]}, otherwise 'row', output df-like records
    format = 'default'
    # i.e, output all gridded points along the line; otherwise 'point', output only end-points.
    zmode = 'line'
    if mode is not None:
        if 'point' in mode.lower():
            zmode = 'point'
        if 'row' in mode.lower():
            format = 'row'
    #print("current mode: ", format, " & ", zmode)

    if len(lonx) != len(latx):
        ds.close()
        return JSONResponse(status_code=status.HTTP_400_BAD_REQUEST,
                            content=jsonable_encoder({"Error": "Check your input of lon/lat should be in equal length"}))

    # May not have exact lon, lat in gridded lon-lat, so need to calculate index
    elif len(lonx) == 1:
        # st = time.time()
        lon0 = gridded_arcsec(lonx[0], basex, arc)
        lat0 = gridded_arcsec(latx[0], basey, arc)
        mlon0x = ds["lon"][lon0]
        mlat0x = ds["lat"][lat0]
        st1 = ds.sel(lon=mlon0x, lat=mlat0x)
        ds.close()
        loc1 = [lonx[0], latx[0]]
        xt1 = np.array([st1['elevation'].values])
        if format == 'row':
            # df1= pd.DataFrame({"longitude": np.array([loc1[0]]).tolist(),
            df1 = pl.DataFrame({"longitude": np.array([loc1[0]]).tolist(),
                                "latitude": np.array([loc1[1]]).tolist(),
                                "z": xt1.tolist(), "distance": np.array([0]).tolist()},
                               columns=['longitude', 'latitude', 'z', 'distance'])
        else:
            out = jsonable_encoder({"longitude": np.array([loc1[0]]).tolist(),
                                    "latitude": np.array([loc1[1]]).tolist(),
                                    "z": xt1.tolist(), "distance": np.array([0]).tolist()})
        # et = time.time()
        # print('Get only one-point exe time: ', et-st, 'sec')
        # return(xt1)
    else:
        # st = time.time()
        idx1 = np.empty(shape=[0, 2], dtype=np.int16)
        loc1 = np.empty(shape=[0, 2], dtype=float)
        dis1 = np.array([0.0])
        mlon0 = np.min(lonx)
        #mlon1 = np.max(lonx)
        mlat0 = np.min(latx)
        #mlat1 = np.max(latx)
        # if do subsetting dataset, reference-0-x,y should be biased
        mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0
        mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
        # index from gridded_arcsec will change is reference is ds_s1, not ds
        #print(mlonbase, mlatbase)
        #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
        #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
        for i in range(len(lonx)-1):
            lonidx0 = gridded_arcsec(lonx[i], basex, arc)
            latidx0 = gridded_arcsec(latx[i], basey, arc)
            lonidx1 = gridded_arcsec(lonx[i+1], basex, arc)
            latidx1 = gridded_arcsec(latx[i+1], basey, arc)
            if zmode == 'point':
                idx1 = np.append(
                    idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
                dist = geodesic((latx[i], lonx[i]),
                                (latx[i+1], lonx[i+1])).km
                dis1 = np.append(dis1, dist, axis=None)
                if i == len(lonx)-2:
                    idx1 = np.append(
                        idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[lonx[i+1], latx[i+1]]], axis=0)
            elif lonidx0 == lonidx1 and latidx0 == latidx1:
                idx1 = np.append(
                    idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
                # to match the same length
                if i >= 1:
                    dis1 = curDist(loc1, dis1)
            else:
                if lonx[i] == lonx[i+1]:
                    stepi = -1 if latidx0 > latidx1 else 1
                    rngi = range(latidx0, latidx1+stepi, stepi)
                    leni = len(rngi)
                    for k, y in enumerate(rngi):
                        locy0 = y/arc - basey
                        locy0i = int(locy0)
                        doty0i = locy0-locy0i-0.25/arc  # a small bias to make sure it's in grid
                        locy1 = locy0i + doty0i
                        if (k < (leni-1)) or (stepi == 1 and locy1 < latx[i+1]) or (stepi == -1 and locy1 > latx[i+1]):
                            idx1 = np.append(
                                idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                            loc1 = np.append(loc1, [[lonx[i], locy1]], axis=0)
                            if i >= 1 or k >= 1:
                                dis1 = curDist(loc1, dis1)

                elif latx[i] == latx[i+1]:
                    stepi = -1 if lonidx0 > lonidx1 else 1
                    rngi = range(lonidx0, lonidx1+stepi, stepi)
                    leni = len(rngi)
                    for k, x in enumerate(rngi):
                        locx0 = x/arc - basex
                        locx0i = int(locx0)
                        dotx0i = locx0-locx0i-0.25/arc  # a small bias to make sure it's in grid
                        locx1 = locx0i + dotx0i
                        if (k < (leni-1)) or (stepi == 1 and locx1 < lonx[i+1]) or (stepi == -1 and locx1 > lonx[i+1]):
                            idx1 = np.append(
                                idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                            loc1 = np.append(loc1, [[locx1, latx[i]]], axis=0)
                            if i >= 1 or k >= 1:
                                dis1 = curDist(loc1, dis1)

                else:
                    m = (latx[i+1]-latx[i])/(lonx[i+1]-lonx[i])
                    b = latx[i] - m * lonx[i]  # y = mx + b
                    # print("m, b:", m ,b)
                    if np.absolute(m) <= 1:
                        lidx0 = lonidx0
                        lidx1 = lonidx1
                    else:
                        lidx0 = latidx0
                        lidx1 = latidx1

                    stepi = -1 if lidx0 > lidx1 else 1
                    rngi = range(lidx0, lidx1+stepi, stepi)
                    leni = len(rngi)
                    for k, s in enumerate(rngi):
                        if k == 0:  # should consider internal node not repeated twice
                            idx1 = np.append(
                                idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                            loc1 = np.append(
                                loc1, [[lonx[i], latx[i]]], axis=0)
                            if i >= 1:
                                dis1 = curDist(loc1, dis1)

                        else:
                            if np.absolute(m) <= 1:
                                locx0 = s/arc - basex
                                locx0i = int(locx0)
                                dotx0i = locx0-locx0i-0.25/arc  # a small bias to make sure it's in grid
                                locx1 = locx0i + dotx0i
                                locy1 = m * locx1 + b
                                if (k < (leni-1)) or (stepi == 1 and locx1 < lonx[i+1]) or (stepi == -1 and locx1 > lonx[i+1]):
                                    y = gridded_arcsec(locy1, basey, arc)
                                    idx1 = np.append(
                                        idx1, [[y-mlatbase, s-mlonbase]], axis=0)
                                    loc1 = np.append(
                                        loc1, [[locx1, locy1]], axis=0)
                                    dis1 = curDist(loc1, dis1)
                            else:
                                locy0 = s/arc - basey
                                locy0i = int(locy0)
                                doty0i = locy0-locy0i-0.25/arc
                                locy1 = locy0i + doty0i
                                locx1 = (locy1 - b)/m
                                if (k < (leni-1)) or (stepi == 1 and locy1 < latx[i+1]) or (stepi == -1 and locy1 > latx[i+1]):
                                    x = gridded_arcsec(locx1, basex, arc)
                                    idx1 = np.append(
                                        idx1, [[s-mlatbase, x-mlonbase]], axis=0)
                                    loc1 = np.append(
                                        loc1, [[locx1, locy1]], axis=0)
                                    dis1 = curDist(loc1, dis1)

                if i == len(lonx)-2:
                    idx1 = np.append(
                        idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[lonx[i+1], latx[i+1]]], axis=0)
                    dis1 = curDist(loc1, dis1)

        # np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f')
        # st = time.time()
        # mlon0 = np.min(lonx) #may cause slice offset to mlonbase, an offset +-1
        mlon1 = np.max(lonx) + 1.5/arc  # to make it larger
        # mlat0 = np.min(latx) #may cause slice offset to mlatbase, an offset +-1
        mlat1 = np.max(latx) + 1.5/arc
        mlon0x = ds["lon"][mlonbase]
        mlat0x = ds["lat"][mlatbase]
        ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(
            mlat0x, mlat1)) if subsetFlag else ds
        ds.close()
        # print(ds_s1['elevation'].shape)
        xt1 = ds_s1['elevation'].values[tuple(idx1.T)]
        ds_s1.close()
        if format == 'row':
            # df1= pd.DataFrame({"longitude": loc1[:, 0].tolist(),
            df1 = pl.DataFrame({"longitude": loc1[:, 0].tolist(),
                                "latitude": loc1[:, 1].tolist(),
                                "z": xt1.tolist(),
                                "distance": dis1.tolist()},
                               columns=['longitude', 'latitude', 'z', 'distance'])
        else:
            out = jsonable_encoder({"longitude": loc1[:, 0].tolist(),
                                    "latitude": loc1[:, 1].tolist(),
                                    "z": xt1.tolist(),
                                    "distance": dis1.tolist()})
        # et = time.time()
        # print('Get value from index of points, time: ', et-st, 'sec')
        # return({"data": xt1})
        # np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

    # st = time.time()
    # jt1 = orjson.dumps(xt1.tolist(), option=orjson.OPT_NAIVE_UTC |
    #                   orjson.OPT_SERIALIZE_NUMPY)
    # out = jsonable_encoder({"data": xt1.tolist()})
    if format == 'row':
        # out = df1.to_dict(orient='records') #by using pandas
        # out = df1.to_dicts()  # by polars
        out = df1
    # et = time.time()
    # print('4 Convert JSON by fastapi: ', et-st, 'sec')
    return out #JSONResponse(content=out)


In [18]:
px = zprofile(lonx, latx, mode='row')
print(px)

shape: (104, 4)
┌───────────┬───────────┬───────┬──────────┐
│ longitude ┆ latitude  ┆ z     ┆ distance │
│ ---       ┆ ---       ┆ ---   ┆ ---      │
│ f64       ┆ f64       ┆ i64   ┆ f64      │
╞═══════════╪═══════════╪═══════╪══════════╡
│ -0.15346  ┆ -0.163806 ┆ -4856 ┆ 0.0      │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ -0.150722 ┆ -0.159375 ┆ -4852 ┆ 0.577017 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ -0.148148 ┆ -0.155208 ┆ -4852 ┆ 0.542563 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ -0.145574 ┆ -0.151042 ┆ -4851 ┆ 0.542563 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ ...       ┆ ...       ┆ ...   ┆ ...      │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 0.1416    ┆ 0.253125  ┆ -4961 ┆ 0.572342 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 0.144651  ┆ 0.257292  ┆ -4964 ┆ 0.572342 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 0.147701  ┆ 0.261458  ┆ -4966 ┆ 0.572342 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 0.149808  ┆ 0.264336  ┆ -4969 ┆ 0.395